<a href="https://colab.research.google.com/github/cedenoruel/DropletNucleation/blob/main/EvapModel_CNT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
 
import scipy 
from scipy import signal
 
from scipy.signal import find_peaks, peak_prominences
from scipy import integrate
!pip install lmfit
import lmfit
from scipy.integrate import odeint
from scipy.integrate import quad as sci_quad
#from scipy.integrate import quad
from scipy.signal import savgol_filter



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:


def find_radius(V,theta):
  return ((V)/(np.pi*g_theta_func(theta)))**(1/3)


def curve_fit_func(func,data_x,data_y,parameters,init_values,method):
  from lmfit import Model, Parameter, report_fit

  Function =  Model(func)
  params = Function.make_params()

  for i in range(0,len(init_values)):
    params[str(parameters[i])].value = init_values[i]

  result = Function.fit(data_y, params, x=data_x,nan_policy='omit',method=method )
  result.plot()
  print(result.fit_report())
  
  values = np.array([result.params[str(parameters[i])].value for i in range(0,len(init_values))])
  stderr = np.array([result.params[str(parameters[i])].stderr for i in range(0,len(init_values))])
  return values, stderr


class DropletEvap:
  #Physical Constants
  T_amb = 298 #ambient temperature in K
  k_b = 1.38064852e-23 # m 2 kg s -2 K -1 ; boltzmann constant
  R = 8.3145 # Pa.m^3/mol.K  #ideal gas constant

  #System Properties (Default is NaCl-water-PDMS system)
  b_1 = 0.205 # density_change_coefficient ; rho = rho_0*(1+b_1*S)   #fitted from https://handymath.com/cgi-bin/nacltble.cgi?submit=Entry
  b_2 = 0.225 #  vapor pressure lowering coefficient ; RH = 1 - b_2*S  #fitted from  https://cdnsciencepub.com/doi/pdf/10.1139/v78-301
  D_ab = 6.736249999999999e-09 #diffusivity of water in oil in m^2/s
  M_w = 0.018 # kg/mol #molecular mass of pure water
  M_nacl = 0.0584 #molar mass of solute in kg/mol  
  rho = 998  #density of pure water # kg/m^3
  c_sat = 8.76 # solubility of water in PDMS #mol per m^3 
  c_eq = 6.14 #solubility of solute in water mol/kg nacl at saturation
  number_of_ions = 2 #ions in one formula unit
  number_density = 2.226e28  # number/m^3 for NaCl calculated from molar mass and avogadros number
  K_evap = D_ab*c_sat*M_w/(rho) # unit in 1/s  #overall mass transfer coefficient of  evaporation #based on solubility of water in PDMS
  init_theta = 105*np.pi/180 #default contact angle
  theta_receding = 86*np.pi/180   #default receding angle for stick-slide mode
  match_supersat = 1.3952400443073647 #supersaturation at matching 

  #Default Settings
  density_corr = True
  oil_height_corr = True
  water_activity_corr = True
  RH_corr = True
  S_0 = 1
  contact_line_behavior = 'CCA'
  n_x = 2 #number of interacting neighbor
  n_y = 1 #number of interacting rows
  


  def __init__(self,V_s,RH_inf,time_increment,L_x,L_y,time_end,properties):
    #initialize variables
    
    self.V_s = V_s
    self.RH_inf = RH_inf
    self.time_increment = time_increment
    self.L_x = L_x #L/R_0 between droplets
    self.L_y = L_y #L/R_0 between droplet lines
    self.time_end = time_end
    self.h_hat = 0.8e-3/find_radius(V_s,self.init_theta)
    prop_list = ['b_1','b_2','D_ab', 'M_w', 'M_nacl', 'rho', 'c_sat', 'c_eq', 'number_of_ions', 'number_density','init_theta', 'match_supersat'] 
    for i in range(0,len(prop_list)):
      setattr(self,prop_list[i],properties[i])

  def update_system_properties(properties):  #for systems other than NaCl-water-PDMS, modify by passing a list of values corresponding to prop_list
    prop_list = ['b_1','b_2','D_ab', 'M_w', 'M_nacl', 'rho', 'c_sat', 'c_eq', 'number_of_ions', 'number_density','init_theta', 'match_supersat'] 
    for i in range(0,len(prop_list)):
      setattr(self,prop_list[i],properties[i])


  #def RH_correction(self,S_0,  init_theta, V_d0, RH_inf, theta,y,L_x,L_y,n_x,n_y):  # Masoud et al. doi:10.1017/jfm.2021.785
  def RH_correction(self,y,theta): # Masoud et al. doi:10.1017/jfm.2021.785
    S_0 = self.S_0
    init_theta = self.init_theta
    V_d0 = self.V_s/S_0
    RH_inf = self.RH_inf
    L_x =self.L_x
    L_y=self.L_y
    n_x = self.n_x
    n_y = self.n_y
    #V_0 = Vd_to_V(V_d0, S_0)
    V_d0 = np.array(V_d0)
    L = L_x #distance between adjacent droplet
    L_2 = L_y #distance between adjacent rows
    y = np.array(y)
    theta = np.array(theta)

    def find_A(theta):   
      def integrand_A(tau,theta):
        return (1+np.cosh((2*np.pi-theta)*tau)/(np.cosh(theta*tau)))**(-1)
      return np.array(sci_quad(integrand_A, 0, 100, args=(theta))[0])

    def find_B(theta):   
      def integrand_B(tau,theta):
        return (1+np.cosh((2*np.pi-theta)*tau)/(np.cosh(theta*tau)))**(-1)*tau**2
      return np.array(sci_quad(integrand_B, 0, 100, args=(theta))[0])  

    r_0 = (V_d0/(np.pi*g_theta_func(init_theta)))**(1/3)

    A = find_A(theta) #parameter equation 3.1 of doi:10.1017/jfm.2021.785
    B = find_B(theta) #parameter equation 3.1 of doi:10.1017/jfm.2021.785
    R = ((y*V_d0/(np.pi*g_theta_func(init_theta))))**(1/3) #time dependent contact radius
    z_hat = R/np.sin(theta)*(1-np.cos(theta))  #droplet height

    r_hat = np.sqrt((L*r_0)**2+z_hat**2) #direct neighbor distance    
    phi_hat = 4*A*R/r_hat+(A-4*B)*R**3*(r_hat**2-3*z_hat**2)/(r_hat)**5

    def find_r_hat(L,r_0,z_hat,n_x):
      n = n_x
      return np.sqrt((n*(L*r_0))**2+z_hat**2)
    def find_r_hat_2(L,L_2,r_0,z_hat,n_x,n_y): #for 2D case
      L_diag = np.sqrt((n_x*(L*r_0))**2+(n_y*(L_2*r_0))**2)
      return np.sqrt(((L_diag))**2+z_hat**2)

    def find_phi_hat(r_hat):
      return 4*A*R/r_hat+(A-4*B)*R**3*(r_hat**2-3*z_hat**2)/(r_hat)**5

    def RH_correction_vs_neighbor_number(n_x,n_y):
      r_hat_array = np.array([find_r_hat(L,r_0,z_hat,i) for i in range(1,int(n_x+1))])    
      phi_hat_array = np.array([find_phi_hat(i) for i in r_hat_array])

      r_hat_array_2D = []
      r_hat_aligned = []
      for j in range(1,int(n_y+1)):
        r_hat_array_2 = np.array([find_r_hat_2(L,L_2,r_0,z_hat,i,j) for i in range(1,int(n_x+1))]) #for 2D, L_2 = distance from 1 line to another divided by r0
        r_hat_array_al = np.array([find_r_hat_2(L,L_2,r_0,z_hat,i,j) for i in [0]])
        r_hat_array_2D.extend([r_hat_array_2])
        r_hat_aligned.extend([r_hat_array_al])
      phi_hat_array_2 = np.array([find_phi_hat(i) for i in np.array(r_hat_array_2D).flatten()])  #for 2D
      if n_y > 0:
        phi_hat_array_aligned = np.array([find_phi_hat(i) for i in np.array(r_hat_aligned).flatten()])
      else:
        phi_hat_array_aligned = 0
      return 1/(1+(2*np.sum(phi_hat_array)+2*np.sum(phi_hat_array_aligned)+4*np.sum(phi_hat_array_2)))

    RH_correction = RH_correction_vs_neighbor_number(n_x,n_y)

    return 1-RH_correction*(1-RH_inf), RH_correction




  #def Popov_intern(self,S_0, init_theta, V_d0, RH_inf,density_corr, oil_height_corr, RH_corr, water_activity_corr, contact_line_behavior,time_increment): #to neglect a correction, put 0 , e.g. density_corr=0,  contact_line_behavior = (CCA, CCR, SS)
  def Popov(self):  
    
    #DEFINE VARIABLES    
    S_0 = self.S_0
    init_theta = self.init_theta
    V_d0 = self.V_s
    RH_inf = self.RH_inf
    time_increment = self.time_increment
    contact_line_behavior = self.contact_line_behavior
    time_end = self.time_end
    L_x = self.L_x
    L_y = self.L_y
    n_x = self.n_x
    n_y = self.n_y

    init_V = 1 #V/V_0

    #CORRECTIONS 
    density_coef = self.b_1 if self.density_corr == True else 0
    oil_height_ratio = self.h_hat if self.oil_height_corr == True else  self.h_hat*1e15
    raoult_coef = self.b_2 if self.water_activity_corr == True else 0

    #WATER VOLUME TO DROPLET VOLUME (VICE VERSA)
    def V_to_Vd(V,supersat):
      return (1+c_eq*self.M_nacl*supersat)*V/(1+density_coef*supersat)
    def Vd_to_V(V_d, supersat):
      return (1+density_coef*supersat)*V_d/(1+self.c_eq*self.M_nacl*supersat)
    V_0 = Vd_to_V(V_d0, S_0)
    r_0 = find_radius(V_d0,init_theta)
    init_V = 1  #initial condition for V/V_0



    #DIFFERENTIAL EQUATION SOLVER
    def f(t, z):
      y, theta = z   #y is instantaneous V/V0, z is a time-dependent array containing both y and theta
      y = z[0]  #extract y from z
      #theta = init_theta  #input initial value

      def integrand(j, theta):    #geometric factor due to change in contact angle https://journals.aps.org/pre/abstract/10.1103/PhysRevE.71.036313
        return (4*(1+np.cosh(2*theta*j))*(np.tanh((np.pi-theta)*j)))/(np.sinh(2*np.pi*j))

      f_theta = np.sin(theta)/(1+np.cos(theta))+sci_quad(integrand, 0, 90, args=(theta))[0] 
      g_theta = ((np.cos(theta))**3-3*np.cos(theta)+2)/(3*(np.sin(theta))**3)

      r = V_to_Vd((y*V_0/(np.pi*g_theta)),S_0/y)**(1/3)
    
      RH_inf_eff =  self.RH_correction(y,theta)[0]  if RH_corr == True else RH_inf  
      #RH_inf_eff_set.extend([RH_inf_eff])


      #if the V/V_0  is very small (less than 1% of the original volume), set dV_dt to zero to avoid NaN values    
      if y > 0.001:  
        RH_sat = (1-raoult_coef*S_0/y)
        dV_dt = -K_evap*(np.pi/V_0)*(( RH_sat -RH_inf_eff))*f_theta*r**2*(1/r+1/(2*(oil_height_ratio)*r_0))

      else:
        dV_dt = 0  

      if contact_line_behavior == 'CCA':
        d_theta_dt = 0
      if contact_line_behavior == 'CCR':
        d_theta_dt = (1/y)*(dV_dt)*((1+np.cos(theta))**2)*g_theta
      #theta_running.extend([theta])

      if contact_line_behavior == 'SS':
        d_theta_dt = (1/y)*(dV_dt)*((1+np.cos(theta))**2)*g_theta if theta> theta_receding else 0

      dydt = [dV_dt, d_theta_dt ] 
      return dydt    
    
    # %% Define time spans, initial values, and constants
    tspan = np.linspace(0, time_end, num=int(time_end/time_increment))
    yinit = [init_V, init_theta]
    sol = solve_ivp(lambda t, z: f(t, z), [tspan[0], tspan[-1]], yinit, t_eval=tspan)
    return np.array(sol.t), sol.y.T[:,0], sol.y.T[:,1] # time in s, volume, contact angle

  def get_Peclet(self,V, theta):
    V_s = self.V_s
    S_0 = self.S_0
    #contact_radius = ((V_CCA*V_s/S_0)/(np.pi*g_theta_func(theta_CCA)))**(1/3)  #V_CCA is V/V_0, V_s/S_0 is the initial volume
    contact_radius = find_radius(V*V_s/S_0,theta)
    surface_area = 2*np.pi*contact_radius**2*((1-np.cos(theta))/np.sin(theta))
    evaporation_rate = np.gradient(V*V_s/S_0, time_increment)
    peclet = -2*contact_radius*evaporation_rate/(surface_area*diffusivity_NaCl_in_water)
    return peclet

  def get_RH_eff_set(self,V, theta):
    V_s = self.V_s
    S_0 = self.S_0
    y = V
    RH_inf_eff =  self.RH_correction(y,theta)[0]  if RH_corr == True else RH_inf  
    return RH_inf_eff

  #INITIALIZE CALCULATED RESULTS

  time = np.array([])
  volume = np.array([]) #normalized volume V/V0
  angle = np.array([])
  S = np.array([])
  peclet = np.array([])
  RH_inf_eff_set = np.array([])
  S_nuc = np.array([])
  i_nuc = np.array([])
  t_nuc_norm = np.array([])
  #solve_peclet = True

  def get_solution(self,solve_peclet=False,solve_RH_eff=False):


    self.time, self.volume, self.angle = self.Popov()
    self.S = self.S_0/self.volume
    #solve_Peclet = False
    #solve_RH_eff = False

    self.peclet = self.get_Peclet(self.volume,self.angle) if solve_peclet == True else 'set solve_peclet = True'
    self.RH_inf_eff_set = [self.get_RH_eff_set(i,j) for i, j in zip(self.volume,self.angle)] if solve_RH_eff == True else 'set solve_RH_eff = True'
    #self.peclet = self.get_Peclet(self.volume,self.angle)
    #self.RH_inf_eff_set = [self.get_RH_eff_set(i,j) for i, j in zip(self.volume,self.angle)]

  def get_Snuc(self,t_sat,t_match,t_nuc):
    t_sat, t_match, t_nuc = [np.array(t_sat),np.array(t_match),np.array(t_nuc)]
    tau = np.array((t_nuc - t_sat)/( t_match -t_sat ))
    #print("tau",tau)
    i_sat_pred = find_nearest_index(self.S,1.00)
    i_match_pred = find_nearest_index(self.S,self.match_supersat)
    i_tau_pred = (np.arange(0,len(self.S),1) - i_sat_pred)/(i_match_pred - i_sat_pred)
    
    self.i_nuc = [find_nearest_index(i_tau_pred,i) for i in tau]
    self.t_nuc_norm = self.time_increment*( i_sat_pred +(i_match_pred - i_sat_pred)*np.array([np.arange(0,len(self.S),1)[j] for j in self.i_nuc])) 
    #print("i_nuc",i_nuc)
    S_nuc = np.array([getattr(self, 'S')[j] for j in self.i_nuc])
    self.S_nuc = S_nuc
    return S_nuc

  def CNT(self,S_n,gamma,log_A):
    R_c = 2*gamma/(number_of_ions*k_b*T_amb*number_density*(np.log(activity_coef_ratio(S_n)*S_n)))
    Delta_G_star = (4/3)*np.pi*gamma*R_c**2
    J = 10**log_A*np.exp(-Delta_G_star/(k_b*T_amb))   
    return J

  def P_pred(self,x, gamma, log_A):     
    time = self.time
    S_n = np.sort(self.S_nuc)
    V_n = self.V_s*self.S_0/S_n
    t_n = savgol_filter(self.t_nuc_norm,3,1)
    J = self.CNT(S_n,gamma,log_A)
    
    JV_pred = J*V_n      

    int_JVt_pred = np.array([np.trapz(JV_pred[0:i],x=np.sort(t_n[0:i])) for i in range(0,len(t_n))])
    P_pred = 1- np.exp(-int_JVt_pred)
    return P_pred
  
  def fit_CNT(self,t_sat,t_match, t_nuc,parameters,init_values,method):
    S_n = self.get_Snuc(t_sat,t_match,t_nuc)
    t_pred = self.t_nuc_norm

    val, stderr = curve_fit_func(self.P_pred,data_x=np.sort(S_n),data_y=cum_dist(t_pred),parameters=parameters,init_values=init_values,method=method)


#time_42pL, V_42pL, theta_42pL = drop_42pL.Popov()


